# 并行处理SimpleDirectoryReader


在这个笔记本中，我们演示了如何在使用`SimpleDirectoryReader`加载数据时使用并行处理。并行处理在处理较重的工作负载时非常有用，比如从包含许多文件的目录中加载数据。（注意：如果使用Windows，在使用并行处理加载数据时可能会看到较少的收益。这与Linux/Mac和Windows中多进程工作方式的差异有关，例如参见[这里](https://pythonforthelab.com/blog/differences-between-multiprocessing-windows-and-linux/)或[这里](https://stackoverflow.com/questions/52465237/multiprocessing-slower-than-serial-processing-in-windows-but-not-in-linux)）


In [ ]:
import cProfile, pstats
from pstats import SortKey

在这个演示中，我们将使用来自[llamahub](https://llamahub.ai)的`PatronusAIFinanceBenchDataset`数据集。该数据集基于从llamahub下载的一组包含在32个PDF文件中的数据。


In [ ]:
!llamaindex-cli download-llamadataset PatronusAIFinanceBenchDataset --download-dir ./data

In [ ]:
from llama_index.core import SimpleDirectoryReader

# 使用包含32个pdf文件的目录定义我们的读取器
reader = SimpleDirectoryReader(input_dir="./data/source_files")

### 顺序加载


顺序加载是默认行为，可以通过`load_data()`方法执行。


In [ ]:
documents = reader.load_data()
len(documents)

4306

In [ ]:
cProfile.run("reader.load_data()", "oldstats")
p = pstats.Stats("oldstats")
p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats(15)

Wed Jan 10 12:40:50 2024    oldstats

         1857432165 function calls (1853977584 primitive calls) in 391.159 seconds

   Ordered by: cumulative time
   List reduced from 292 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  391.159  391.159 {built-in method builtins.exec}
        1    0.003    0.003  391.158  391.158 <string>:1(<module>)
        1    0.000    0.000  391.156  391.156 base.py:367(load_data)
       32    0.000    0.000  391.153   12.224 base.py:256(load_file)
       32    0.127    0.004  391.149   12.223 docs_reader.py:24(load_data)
     4306    1.285    0.000  387.685    0.090 _page.py:2195(extract_text)
4444/4306    5.984    0.001  386.399    0.090 _page.py:1861(_extract_text)
     4444    0.006    0.000  270.543    0.061 _data_structures.py:1220(operations)
     4444   43.270    0.010  270.536    0.061 _data_structures.py:1084(_parse_content_stream)
36489963/33454574   32.688    0.000

### 并行加载


要使用并行进程加载数据，我们将`num_workers`设置为一个正整数值。


In [ ]:
documents = reader.load_data(num_workers=10)

In [ ]:
len(documents)

4306

In [ ]:
cProfile.run("reader.load_data(num_workers=10)", "newstats")
p = pstats.Stats("newstats")
p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats(15)

Wed Jan 10 13:05:13 2024    newstats

         12539 function calls in 31.319 seconds

   Ordered by: cumulative time
   List reduced from 212 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   31.319   31.319 {built-in method builtins.exec}
        1    0.003    0.003   31.319   31.319 <string>:1(<module>)
        1    0.000    0.000   31.316   31.316 base.py:367(load_data)
       24    0.000    0.000   31.139    1.297 threading.py:589(wait)
       23    0.000    0.000   31.139    1.354 threading.py:288(wait)
      155   31.138    0.201   31.138    0.201 {method 'acquire' of '_thread.lock' objects}
        1    0.000    0.000   31.133   31.133 pool.py:369(starmap)
        1    0.000    0.000   31.133   31.133 pool.py:767(get)
        1    0.000    0.000   31.133   31.133 pool.py:764(wait)
        1    0.000    0.000    0.155    0.155 context.py:115(Pool)
        1    0.000    0.000    0.155    0.155 pool

### 总结


In [ ]:
391 / 30

13.033333333333333

正如上面的结果所示，当从包含许多文件的目录中加载数据时，使用并行处理可以实现大约13倍的加速（或速度提高1200%）。
